In [10]:
from datasets import load_dataset
import os

ds = load_dataset("hotchpotch/JQaRA", split="test")

In [29]:


# 保存先ディレクトリ
output_dir = "./jqara_data"
os.makedirs(output_dir, exist_ok=True)

# 出力ファイルパス
output_path = os.path.join(output_dir, "jqara_test.jsonl")

# JSONL 形式で保存
ds.to_json(output_path, orient="records", lines=True, force_ascii=False)

print(f"✅ Saved to {output_path}")


Creating json from Arrow format: 100%|██████████| 167/167 [00:00<00:00, 179.65ba/s]

✅ Saved to ./jqara_data/jqara_test.jsonl


In [23]:
# 最初は負荷軽減のため N 件だけ
N = int(os.getenv("LIMIT", "5000"))
subset = ds.select(range(min(N, len(ds))))

In [1]:
import weaviate

client = weaviate.connect_to_local()


In [6]:
CLASS_NAME="jqara_colbert"

if client.collections.exists(CLASS_NAME):
    client.collections.delete(CLASS_NAME)

In [7]:
from weaviate.classes.config import Configure, Property, DataType

client.collections.create(
    name=CLASS_NAME,
    vector_config=[
        # User-provided embeddings
        Configure.MultiVectors.self_provided(
            name="multi_vector",
        ),
    ],
    properties=[
        Property(name="q_id", data_type=DataType.TEXT),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)

In [4]:
from weaviate.classes.config import Configure, Property, DataType

MUVERA_CLASS_NAME = "jqara_muvera"

client.collections.create(
    name=MUVERA_CLASS_NAME,
    vector_config=[
        # User-provided embeddings
        Configure.MultiVectors.self_provided(
            name="multi_vector",
            encoding=Configure.VectorIndex.MultiVector.Encoding.muvera(),
        ),
    ],
    properties=[
        Property(name="q_id", data_type=DataType.TEXT),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)

In [7]:
from colbert.infra import ColBERTConfig
from colbert.modeling.checkpoint import Checkpoint

ckpt = Checkpoint("jinaai/jina-colbert-v2", colbert_config=ColBERTConfig())

/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


[Nov 08, 14:30:03] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarni

In [8]:
def to_passage(row): 
    return f"{row['title']}\n{row['text']}" if row['title'] else row['text']

In [24]:
texts = [to_passage(r) for r in ds]
ids = list(range(len(texts)))
embeddings = ckpt.docFromText(texts, bsize=8)

/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


KeyboardInterrupt: 

In [26]:
from tqdm import tqdm
import torch

# ColBERTモデルからembeddingを取得（進捗バー付き）
bsize = 8
embeddings = []  # 各バッチの結果を順次蓄積

# tqdmのprogress bar
for i in tqdm(range(0, len(texts), bsize), desc="🔍 Encoding documents with ColBERT"):
    batch = texts[i:i + bsize]

    # ColBERTでバッチ埋め込み生成
    embs = ckpt.docFromText(batch, bsize=bsize, keep_dims=True)

    # docFromTextがtupleを返す場合は中身を取り出す
    if isinstance(embs, tuple):
        embs = embs[0]

    # GPUならCPUへ移動しておく
    if isinstance(embs, torch.Tensor) and embs.is_cuda:
        embs = embs.cpu()

    embeddings.append(embs)

# 最後に全バッチを連結して [N, L, H] に
embeddings = torch.cat(embeddings, dim=0)

print("✅ Done:", embeddings.shape)


🔍 Encoding documents with ColBERT:   4%|▍         | 875/20838 [13:39<5:11:28,  1.07it/s]


KeyboardInterrupt: 

In [28]:
from tqdm import tqdm
import torch
from colbert.infra import ColBERTConfig
from colbert.modeling.checkpoint import Checkpoint

# ====== 推奨ハイパラ（調整可） ======
DOC_MAXLEN = 256          # 512→256に落とすとだいたい倍速（精度と相談）
BSIZE = 24                # CPUなら 8〜32 を目安に
NUM_THREADS = max(1, int(torch.get_num_threads() or 1))

# AppleシリコンでCPU最適化（Accelerateを利用）
torch.set_num_threads(NUM_THREADS)          # 演算スレッド
torch.set_num_interop_threads(1)            # スレッド間調停は小さく
# ↑ 遅い場合は NUM_THREADS を os.cpu_count()-2 くらいにして試す

cfg = ColBERTConfig(doc_maxlen=DOC_MAXLEN)
ckpt = Checkpoint("jinaai/jina-colbert-v2", colbert_config=cfg)

# ColBERT内部モデルをeval
try:
    ckpt.model.eval()
except Exception:
    pass

def trim_pad(emb_2d: torch.Tensor):
    # [L,H] のゼロ行（パディング）を落とす
    mask = ~(emb_2d == 0).all(dim=1)
    return emb_2d[mask]

pairs = []   # (id, vectors[list[list[float]]]) を貯める

with torch.inference_mode():
    for i in tqdm(range(0, len(texts), BSIZE), desc="🔍 ColBERT CPU encoding"):
        batch = texts[i:i+BSIZE]

        embs = ckpt.docFromText(batch, bsize=BSIZE, keep_dims=True)
        if isinstance(embs, tuple):
            embs = embs[0]               # [B,L,H]

        # CPU前提だが一応
        if isinstance(embs, torch.Tensor) and embs.is_cuda:
            embs = embs.cpu()

        # 各ドキュメントごとに [L,H] を処理
        for j, emb2d in enumerate(embs):
            pruned = trim_pad(emb2d)
            doc_id = i + j
            pairs.append((doc_id, pruned.tolist()))

print(f"✅ done. encoded docs: {len(pairs)}")


/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(
🔍 ColBERT CPU encoding:   0%|          | 15/6946 [00:44<5:41:15,  2.95s/it]


KeyboardInterrupt: 

In [14]:
print(embeddings[0])
print(ids)

tensor([[[ 0.0131,  0.1063, -0.1633,  ..., -0.0732,  0.1433,  0.0374],
         [ 0.1378,  0.0773, -0.1280,  ..., -0.0815,  0.0022, -0.0669],
         [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         ...,
         [ 0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.0000]],

        [[ 0.0309,  0.1158, -0.1871,  ..., -0.0918,  0.1124, -0.0211],
         [ 0.1450,  0.0748, -0.1341,  ..., -0.0841,  0.0123, -0.0684],
         [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         ...,
         [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         [ 0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000]],

        [[ 0.0133,  0.0954, -0.1811,  ..., -0.0788,  0.1357,  0.0269],
         [ 0.1420,  0.0770, -0.1307,  ..., -0

In [3]:
CLASS_NAME="jqara_colbert"

colbert_col = client.collections.get(CLASS_NAME)
muvera_col = client.collections.get(MUVERA_CLASS_NAME)
# col = client.collections.get(CLASS_NAME)
col = muvera_col

NameError: name 'MUVERA_CLASS_NAME' is not defined

In [15]:
from tqdm import tqdm



total = len(texts)

# tqdmで進捗バーを表示
with col.batch.fixed_size(batch_size=16) as batch, tqdm(total=total, desc="📤 Uploading to Weaviate") as pbar:
    for i, (text, emb) in enumerate(zip(texts, embeddings[0])):

        # オブジェクトを追加
        batch.add_object(
            properties={
                "text": text,
                "q_id": subset[i]["q_id"],
                "title": subset[i]["title"],
            },
            vector={"multi_vector": emb.tolist()},  # ★v4では vectors= を使う
        )

        # 進捗更新
        pbar.update(1)

        # 10件ごとに失敗件数を確認して表示（任意）
        if (i + 1) % 10 == 0 and col.batch.failed_objects:
            print(f"⚠️ 途中までの失敗件数: {len(col.batch.failed_objects)}")

# 全体完了後のチェック
if col.batch.failed_objects:
    print(f"\n❌ Number of failed imports: {len(col.batch.failed_objects)}")
    print(f"First failed object: {col.batch.failed_objects[0]}")
else:
    print("\n✅ All objects uploaded successfully!")

# コレクション内の件数を確認
count = col.aggregate.over_all(total_count=True).total_count
print(f"🧮 現在の登録件数: {count}")


📤 Uploading to Weaviate: 100%|██████████| 5000/5000 [00:16<00:00, 304.40it/s]



✅ All objects uploaded successfully!
🧮 現在の登録件数: 5000


In [22]:
from weaviate.classes.query import MetadataQuery


query = subset[100]["question"]
q_vecs = ckpt.queryFromText([query], bsize=1)[0]  # [Lq, dim]
print(query)
print(q_vecs)

response = col.query.near_vector(
    near_vector=q_vecs.tolist(),
    target_vector="multi_vector",
    return_metadata=MetadataQuery(distance=True, certainty=True, score=True)
)

for o in response.objects:
    title = (o.properties or {}).get("title")
    text = (o.properties or {}).get("text") or ""
    # near_text なら通常 distance / certainty が入る（要求した場合）
    dist = getattr(o.metadata, "distance", None)
    cert = getattr(o.metadata, "certainty", None)
    score = getattr(o.metadata, "score", None)  # ハイブリッド/BM25で付くことがある
    # どれが取得できたかで柔軟に表示
    if dist is not None:
        metric = f"distance={dist:.4f}"
    elif score is not None:
        metric = f"score={score:.4f}"
    elif cert is not None:
        metric = f"certainty={cert:.4f}"
    else:
        metric = "(no metric)"

    print(f"- {metric}  title: {title}")
    print(f"  text: {text[:120]}...")
    print(f"  q_id: {(o.properties or {}).get('q_id')}, answers: {(o.properties or {}).get('answers')}\n")


『傘を持たない蟻たちは』『ピンクとグレー』などの小説を書いた、アイドルグループNEWSのメンバーは誰でしょう?
tensor([[ 0.0669,  0.1357, -0.2555,  ..., -0.0179,  0.0809,  0.0090],
        [ 0.1482,  0.0936,  0.0277,  ...,  0.0062,  0.1405, -0.1709],
        [ 0.0536,  0.0813, -0.0395,  ..., -0.0579, -0.0178, -0.1177],
        ...,
        [ 0.2089,  0.0950, -0.1694,  ...,  0.0155,  0.1996,  0.0767],
        [ 0.1287,  0.1411, -0.0919,  ..., -0.0127,  0.1797, -0.0078],
        [ 0.0735,  0.1379, -0.2705,  ..., -0.0366,  0.0626,  0.0062]])
- distance=-20.6804  title: 加藤シゲアキ
  text: 加藤シゲアキ
加藤 シゲアキ(かとう シゲアキ、1987年〈昭和62年〉7月11日 - )は、日本の男性歌手、俳優、タレント、小説家、劇作家。男性アイドルグループ・NEWSのメンバー。旧芸名・本名は加藤 成亮(読みは同じ)。広島県広島市生ま...
  q_id: QA20CAPR-1006, answers: None

- distance=-20.5510  title: 加藤シゲアキ
  text: 加藤シゲアキ
鬱屈した感情をかたちにしなければという思いと、NEWSのために自分にも何かできるのではないかという思いで、もともと執筆の仕事に定評があったことから小説を書き上げた。抱え込んでしまう感情を小説で形にすることができたこと、多くの反...
  q_id: QA20CAPR-1006, answers: None

- distance=-20.2890  title: ピンクとグレー
  text: ピンクとグレー
ストーリーはアニメーション映画の『PERFECT BLUE』から着想を得ており、結論を見せてから

In [2]:
result = col.aggregate.over_all(total_count=True)

print("📊 登録ドキュメント数:", result.total_count)

NameError: name 'col' is not defined

In [31]:
import torch
data = torch.load("outputs/batch_00000.pt")
data

{'ids': ['QA20CAPR-1004#5191928',
  'QA20CAPR-1004#60221',
  'QA20CAPR-1004#60220',
  'QA20CAPR-1004#60222',
  'QA20CAPR-1004#2665789',
  'QA20CAPR-1004#49531',
  'QA20CAPR-1004#1912169',
  'QA20CAPR-1004#49522',
  'QA20CAPR-1004#298370',
  'QA20CAPR-1004#56160',
  'QA20CAPR-1004#1953461',
  'QA20CAPR-1004#49532',
  'QA20CAPR-1004#59007',
  'QA20CAPR-1004#1459483',
  'QA20CAPR-1004#192230',
  'QA20CAPR-1004#526125',
  'QA20CAPR-1004#431025',
  'QA20CAPR-1004#4935558',
  'QA20CAPR-1004#57203',
  'QA20CAPR-1004#49519',
  'QA20CAPR-1004#2503329',
  'QA20CAPR-1004#135803',
  'QA20CAPR-1004#220168',
  'QA20CAPR-1004#1767473',
  'QA20CAPR-1004#1039044',
  'QA20CAPR-1004#49523',
  'QA20CAPR-1004#1653211',
  'QA20CAPR-1004#2665788',
  'QA20CAPR-1004#4912083',
  'QA20CAPR-1004#3286601',
  'QA20CAPR-1004#298375',
  'QA20CAPR-1004#4126185',
  'QA20CAPR-1004#3114717',
  'QA20CAPR-1004#210425',
  'QA20CAPR-1004#61580',
  'QA20CAPR-1004#43830',
  'QA20CAPR-1004#1046454',
  'QA20CAPR-1004#220130',
  